In [2]:
!pip install tweepy
!pip install jsonpickle

  Using cached jsonpickle-1.4.1-py2.py3-none-any.whl (36 kB)


In [3]:
import tweepy
import jsonpickle
import time
import pandas as pd
import json
import csv

In [4]:
#save files in this folder
datafolder_e = r'D:\18-DS\github\SDSHL\data\external'

# Create convert json to csv

In [5]:
def createCSV_fromJson(csv_fname, txt_fname):
    f = open(csv_fname,'a',encoding='utf8')
    csvWriter = csv.writer(f)
    headers=['full_text','retweet_count','user_followers_count','favorite_count','place','coordinates','geo','created_at','id_str']
    #headers=['full_text']
    csvWriter.writerow(headers)

    for inputFile in [txt_fname]:#all the text-file names you want to convert to Csv in the sae folder as this code
        tweets = []
        for line in open(inputFile, 'r'):
            tweets.append(json.loads(line))

        
        count_lines=0
        for tweet in tweets:
            try:
                csvWriter.writerow([tweet['full_text'],tweet['retweet_count'],tweet['user']['followers_count'],tweet['favorite_count'],tweet['place'],tweet['coordinates'],tweet['geo'],tweet['created_at'],str(tweet['id_str'])])
                #csvWriter.writerow(tweet['full_text'])
                count_lines+=1
            except Exception as e:
                print(e)
        print(count_lines)

# Connect to Twitter using Tweepy api

In [8]:
# Variables that contains the credentials to access Twitter API
ACCESS_TOKEN = '1683653820-p2Ehkyl2OQUmaY7Xf7H9pc4ErNvyvovBVwxy9GC'
ACCESS_SECRET = 'TbtPT96BPIc31FK21b1iX9hcLfg4XOqiRZWrq5gAERoPc'

#CONSUMER_KEY / CONSUMER_SECRET
API_KEY = 'kUHB8y0fHK5WiGapp4CffpP2g'
API_SECRET ='Gd614dFtCjGWwgN9M4t6jVT83q5kntkyDFDRmpZE8D7JfGBLI7'

# Setup access to API
def connect_to_twitter_OAuth():
    auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    api = tweepy.API(auth)
    return api


In [9]:
# Create API object
api = connect_to_twitter_OAuth()
status = "Testing!"
#api.update_status(status=status)

# tweets from my stream

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

# CreateJsonFile_ofTweets

In [10]:
def CreateJsonFile_ofTweets(fname, hashtag, max_tweets=1000000):
    #auth = tweepy.AppAuthHandler(API_KEY,API_SECRET)
    #api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
    
    api = connect_to_twitter_OAuth()
    #wait_on_rate_limit – Whether or not to automatically wait for rate limits to replenish
    #wait_on_rate_limit_notify – Whether or not to print a notification when Tweepy is waiting for rate limits to replenish
    # the sleep mode is automatically enabled with above 2 args

    tweetsPerQuery = 100#this is the maximum provided by API
    #max_tweets just for the sake of While loop

    # No sinceId and max_id ..Get whathever you have exhaustively
    since_id = None
    max_id = -1
    tweet_count = 0
    print("Downloading the tweeets..takes some time..")

    search_query=hashtag 
    x=0
    with open(fname,'w') as f:
        print("Downloading hashtag" + search_query)
        while(tweet_count<max_tweets):
            try:
                if(max_id<=0):
                    if(not since_id):
                        new_tweets = api.search(q=search_query,count=tweetsPerQuery,lang="hi",
                                                tweet_mode='extended', include_rts=False)
                    else:
                        new_tweets = api.search(q=search_query,count=tweetsPerQuery,lang="hi",
                                                tweet_mode='extended',since_id=since_id, include_rts=False)
                else:
                    if(not since_id):
                        new_tweets = api.search(q=search_query,count=tweetsPerQuery,lang="hi",
                                                tweet_mode='extended',max_id=str(max_id-1), include_rts=False)
                    else:
                        new_tweets = api.search(q=search_query,count=tweetsPerQuery,lang="hi",tweet_mode='extended',
                                                max_id=str(max_id-1),since_id=since_id, include_rts=False)
                
                # Tweets Exhausted
                if(not new_tweets):
                    print("No more tweets found!!")
                    break
                    
                # write all the new_tweets to a json file
                for tweet in new_tweets:
                    f.write(jsonpickle.encode(tweet._json,unpicklable=False)+'\n')
                    tweet_count+=len(new_tweets)
                    #print("Successfully downloaded {0} tweets".format(tweet_count))
                    max_id=new_tweets[-1].id
                    
            # in case of any error
            except tweepy.TweepError as e:
                    print("Some error!!:"+str(e))
                    break
    return tweet_count

# tweets from a specific user

In [31]:
#tweets = api.user_timeline(account, count=20000, lang="hi", tweet_mode="extended")

In [30]:
#disabled accounts = 'Real_Netan', 'sandeep353055' yatisharma111, Arnab5222, 'PbSwain_IND', DeshbhaktRosha1, ShrishtySharma_

accounts = ['acs_rkpandey','Bittu_Tufani','Gulmoha99443364','maliksarah5gma1','mjaved819',
            'Rajeshkm_RJS','pallavinaharre1','raviag07','samratkikavitae','Sourav1717',
            'Sujitkumar_mau','theBookwalla','gareeb_bhartiya','AkshayM90903668','vptvns',
            'Vyang_Kumar','HindiSatire','Shanu44339200','SwetaSinghAT','VinodRajotiya82',
            'gautam007','Republic_Bharat','VickyAarya007','KapilMishra_IND','VyangyaVahini',
            'DChaurasia2312','WiskyWala','ManojTiwariMP', 'chitraaum','RealPushpendra',
            'RajatSharmaLive','RubikaLiyaquat','TheAbhishek_IND','Real_Sweta','SushantBSinha',
            'VickyAarya007','badri_dk','Abhasin89009555','BhartiyRudr','Jainritesh_rj','TeriJogan','ridhi_bose']


dfs=[]
for account in accounts:
    tweets = api.user_timeline(account, count=20000, lang="hi", tweet_mode="extended")
    tweets = [[account,tweet.full_text] for tweet in tweets]
    #tweets = (tweet for tweet in tweets)
    df = pd.DataFrame( tweets , columns=['account',"Text"])
    dfs.append(df)

for i in range(len(dfs)):
    print (len(dfs[i]))
    if i==0:
        df=dfs[0]
        i=1
    else:
        df=pd.concat( [df, dfs[i] ], ignore_index=True)
        i+=1
    
df.to_csv(datafolder_e + r'\\' + account+".csv")
print ("Saving file "+ datafolder_e + r'\twitter_data.csv")


200
200
199
200
200
200
197
200
198
197
200
200
192
111
200
162
197
191
200
200
200
200
200
199
197
200
198
200
200
198
200
200
200
200
199
200
200
200
200
200
200
199
Saving file D:\18-DS\github\SDSHL\data\external\\ridhi_bose.csv


# tweets from a specific hashtag

In [34]:

hashtags= ["कटाक्ष", "#व्यंग", "व्यंग्य","मोदी_फिरकी_ले_रहा_है"]
#hashtags= ["मोदी_फिरकी_ले_रहा_है"]

for hashtag in hashtags:
    st = time.time()
    fname= hashtag+".txt"
    tweet_count = CreateJsonFile_ofTweets(fname, "#"+hashtag, 1000)
    createCSV_fromJson(hashtag+'.csv', hashtag+'.txt')
    end = time.time()
    print("A total of {0} tweets are downloaded and saved to {1}".format(tweet_count,fname))
    print("Total time taken is ",end-st,"seconds.")
    #df = pd.read_csv(hashtag+'.csv',  usecols=['created_at','full_text'], encoding = 'utf8')
    df = pd.read_csv(hashtag+'.csv',  usecols=['full_text'], encoding = 'utf8')
    
    #df= df.rename(columns={'created_at':'Time','full_text':'Text'})
    df= df.rename(columns={'full_text':'Text'})
    #df=df[ ['Time','Text'] ]
    df=df[ ['Text'] ]
    df.to_csv( datafolder_e + r'\\' + "_"+hashtag+'.csv')
    print(datafolder_e + r"\_"+ hashtag+'.csv'+ " file created")
    print ("=====================>")

100
A total of 10000 tweets are downloaded and saved to कटाक्ष.txt
Total time taken is  2.0661940574645996 seconds.
_कटाक्ष.csv file created
=====================>
60
A total of 3600 tweets are downloaded and saved to #व्यंग.txt
Total time taken is  4.286127328872681 seconds.
_#व्यंग.csv file created
=====================>
100
A total of 10000 tweets are downloaded and saved to व्यंग्य.txt
Total time taken is  4.182105779647827 seconds.
_व्यंग्य.csv file created
=====================>
No more tweets found!!
0
A total of 0 tweets are downloaded and saved to मोदी_फिरकी_ले_रहा_है.txt
Total time taken is  1.3484437465667725 seconds.
_मोदी_फिरकी_ले_रहा_है.csv file created
=====================>


In [35]:
pd.read_csv(datafolder_e + r'\\' + "_"+hashtags[0]+'.csv')

,Unnamed: 0,Text
0,0,"RT @saurv365: भैंस - रोजगार, कोविड महामारी, अर..."
1,1,"भैंस - रोजगार, कोविड महामारी, अर्थव्यवस्था, कि..."
2,2,RT @0007tilak: आज वो भी #DaughtersDay2020 की श...
3,3,RT @AjjuBhaiyaa: मेरी अक्सर लोगों से बनती नहीं...
4,4,मेरी अक्सर लोगों से बनती नहीं \r\nमें कितना भी...
...,...,...
297,297,आज वो भी #DaughtersDay2020 की शुभकामनाओं सहित ...
298,298,बड़ी खबर- दीपिका ने रणवीर से बोला आते समय नीचे...
299,299,RT @AsifRazakhan_: #कटाक्ष\r\n500-500 सौ रूपये...
300,300,#कटाक्ष\r\n500-500 सौ रूपये\r\nऔर एक प्लेट बिर...


df = pd.read_csv(hashtags[0]+'.csv',  encoding = 'utf8')
df

# Remove all @ read text from Tweets

In [8]:
import pandas as pd
import re

In [77]:
len(df)

4096

In [36]:
df.Text

Series([], Name: Text, dtype: object)

'ManojTiwariMP'

In [11]:
df

,Unnamed: 0,account,Text
0,0,VickyAarya007,रागा- पता है मेरे जन्मदिन पे\nकौन पैदा हुआ था ...
1,1,VickyAarya007,जिस तरह राम रहीम के 36 अश्रम\nसील है उसी तरह म...
2,2,VickyAarya007,@mechitraaa @Abhasin89009555 ye to kuchh nhi h...
3,3,VickyAarya007,सुशांत सिंह राजपूत\nआत्महत्या मामले की सुप्रीम...
4,4,VickyAarya007,RT @Abhasin89009555: इन तस्वीरों को देखकर समझि...
...,...,...,...
1995,1995,ShrishtySharma_,RT @RajTiwa56444261: मित्रो आज हमारे विवाह के ...
1996,1996,ShrishtySharma_,RT @its_mahii01: मै और मेरी तन्हाई अक्सर\n।\n।...
1997,1997,ShrishtySharma_,"RT @universe__guru: देश बचा लो साहब ,\nमोहब्बत..."
1998,1998,ShrishtySharma_,RT @Chatterj1Asking: लगता है 1962 का \nहिसाब ब...


In [12]:
def getclean_text(x):
    text_list= x.split(" ")
    clean_text = []
    #don't include account
    pattern = "^[^@]"
    for token in text_list:
        if re.search(pattern, token):
            clean_text.append(token)
    
    #don't include http links
    clean_text1 = []
    pattern = "^[^http]*[^https]"
    for token in clean_text:
        if re.search(pattern, token):
            clean_text1.append(token)
    
    #print (clean_text1)
    clean_text1=" ".join(clean_text1)
    
    clean_text1= clean_text1.replace("\n"," ")
    
    return clean_text1

df1 = df[ ['account','Text'] ]
df1= pd.DataFrame(df1['Text'].apply(lambda x: getclean_text(x)) )

In [13]:
df1['account']=df.account

In [14]:
df1[ ['account','Text'] ].to_csv( datafolder_e + r'\\' + "_"+accounts[0]+".csv", index=False )
df1

,Text,account
0,रागा- पता है मेरे जन्मदिन पे कौन पैदा हुआ था ?...,VickyAarya007
1,जिस तरह राम रहीम के 36 अश्रम सील है उसी तरह मौ...,VickyAarya007
2,ye kuchh nhi me Jane ke liye isse aage bhi bah...,VickyAarya007
3,सुशांत सिंह राजपूत आत्महत्या मामले की सुप्रीम ...,VickyAarya007
4,RT इन तस्वीरों को देखकर समझिए क्यों जरूरत पड़ी...,VickyAarya007
...,...,...
1995,RT मित्रो आज हमारे विवाह के 18 वर्ष पूरे हुए.....,ShrishtySharma_
1996,RT मै और मेरी तन्हाई अक्सर । । । । चाइना वालो...,ShrishtySharma_
1997,"RT देश बचा लो साहब , मोहब्बत तो कभी मिलने से र...",ShrishtySharma_
1998,RT लगता है 1962 का हिसाब बराबर करना भी मोदीजी...,ShrishtySharma_


In [7]:
import time 
from datetime import datetime
import numpy as np
# Fetch tweet ids from a timeline 
# Use these block if this is there is no saved tweets from 
# this account or you dont plan to use since_id functionality 
import time 
from datetime import datetime

counter = 0
timeline_ids = list()
start_time = datetime.now()

user_id='Shanu44339200'

try:
    for status in tweepy.Cursor(api.user_timeline, user_id=user_id).items():
        # process status here
        timeline_ids.append(status.id)
        end_time = datetime.now()
        if np.ceil((end_time - start_time).seconds / 60) >= 12:
            print("Worked for 12 minutes, waiting for 15 minutes now")
            print(datetime.now())
            time.sleep(60 * 15)
            start_time = datetime.now()
            end_time = datetime.now()

        counter += 1
except tweepy.RateLimitError as e:
    print("Rate limit error exceed waiting for 15 secs")
    print(datetime.now())
    print("You may want to save the timeline ids over there, to not to lose them during execution")
    time.sleep(60 * 15)
    start_time = datetime.now()
    end_time = datetime.now()

print("You may want to save the timeline ids over there, to not to lose them during execution")

You may want to save the timeline ids over there, to not to lose them during execution


In [12]:
import time 
from datetime import datetime

# Get the max_id that already  resides in db
max_id_in_town = 1000 #some_max_id
counter = 0
timeline_ids = list()
start_time = datetime.now()
try:
    for status in tweepy.Cursor(api.user_timeline, user_id=user_id, since_id=max_id_in_town).items():
        # process status here
        timeline_ids.append(status.id)
        print (status.created_at, status.text)
        end_time = datetime.now()
        if np.ceil((end_time - start_time).seconds / 60) >= 12:
            print("Worked for 12 mins, waiting for 15 mins now")
            print(datetime.now())
            print("You may want to save the timeline ids over there, to not to lose them during execution")
            time.sleep(60 * 15)
            start_time = datetime.now()
            end_time = datetime.now()

        counter += 1
except tweepy.RateLimitError as e:
    print("Rate limit error exceed waiting for 15 secs")
    print(datetime.now())
    print("You may want to save the timeline ids over there, to not to lose them during execution")
    time.sleep(60 * 15)
    start_time = datetime.now()
    end_time = datetime.now()
print("You may want to save the timeline ids over there, to not to lose them during execution")

2020-06-12 07:52:45 Testing!
2020-05-23 07:41:16 Testing!
2020-05-22 13:26:20 Testing!
2019-09-07 18:09:31 RT @PMBhutan: We are proud of India and its scientists today. Chandrayaan-2 saw some challenges last minute but the courage and hard work y…
2019-09-07 18:09:23 https://t.co/zIUnVdwrXu

#nonviolence #foodchain #survival
2019-09-07 07:26:16 @anandmahindra A true nationalist, patriotic, ethical business professional. India will always remember the way you… https://t.co/RoL5t0aVVs
2019-09-07 07:25:55 @anandmahindra every time I read your tweets. You occupy more space in my my mind and heart. Along with business, you are leader of masses.
2019-09-07 07:20:45 RT @anandmahindra: The communication isn’t lost. Every single person in India can feel the heartbeat of #chandrayaan2 We can hear it whispe…
2019-09-07 07:20:01 RT @RajivMessage: Chidambaram says his "only worry is about the economy." Does he mean his family economy?
https://t.co/R636ZVqwv1
2019-09-05 14:41:02 Causing #superintell

You may want to save the timeline ids over there, to not to lose them during execution


In [ ]:
# Set looping params
start_time = datetime.now()
raw_tweets = list()

for id_ in timeline_ids:
    try:
        res = api.get_status(id=id_, tweet_mode="extended")
        raw_tweets.append({"accident_id": id_,
                           "created_at": res.created_at,
                           "text": res.full_text})
        end_time = datetime.now()
   
        if np.ceil((end_time - start_time).seconds / 60) >= 12:
            print("Worked for 12 mins, waiting for 15 mins now")
            print(datetime.now())
            print("Save tweets somewhere")
            time.sleep(60 * 15)
            start_time = datetime.now()
            end_time = datetime.now()
        del res

    except tweepy.RateLimitError as e:
        print("Rate limit error exceed waiting for 15 secs")
        print(datetime.now())
        print("Save tweets somewhere")

        time.sleep(60 * 15)
        start_time = datetime.now()
        end_time = datetime.now()
print("Save tweets somewhere")